In [2]:
import numpy as np
import netCDF4 as nc

# 读取数据

In [4]:
#input nino12
with nc.Dataset('/Users/rowenxu/LSTM_EA_Monsoon/Data/input/adaptor.mars.internal-1698315429.528404-2819-2-4309fc02-bb4e-4499-a9e3-9e4d13b68b4c.nc','r') as f:
    latitude=f.variables["latitude"][:]
    longitude=f.variables["longitude"][:]
    time=f.variables["time"][:]
    T=f.variables["t"][:]
    uwind=f.variables["u"][:]
    vwind=f.variables["v"][:]


In [5]:
#normalize the data
T_norm_nino34=np.zeros(T.shape)
uwind_norm_nino34=np.zeros(uwind.shape)
vwind_norm_nino34=np.zeros(vwind.shape)
for z in range(2):
    for i in range(len(latitude)):
        for j in range(len(longitude)):
            T_norm_nino34[:,z,i,j]=(T[:,z,i,j]-np.mean(T[:,z,i,j]))/np.std(T[:,z,i,j])
            uwind_norm_nino34[:,z,i,j]=(uwind[:,z,i,j]-np.mean(uwind[:,z,i,j]))/np.std(uwind[:,z,i,j])
            vwind_norm_nino34[:,z,i,j]=(vwind[:,z,i,j]-np.mean(vwind[:,z,i,j]))/np.std(vwind[:,z,i,j])

print(T_norm_nino34)

[[[[-3.11461780e-01 -3.07224354e-01 -3.06453816e-01 ...  9.28979720e-01
     9.18398107e-01  9.13290413e-01]
   [-3.14635213e-01 -3.09175560e-01 -3.05620021e-01 ...  9.62741767e-01
     9.49742367e-01  9.45176355e-01]
   [-3.20694848e-01 -3.19871173e-01 -3.17883889e-01 ...  9.84765340e-01
     9.76426593e-01  9.73049153e-01]
   ...
   [-6.70545089e-01 -6.71115726e-01 -6.62929194e-01 ...  4.75610453e-01
     4.60209841e-01  4.43569465e-01]
   [-6.77270847e-01 -6.79597929e-01 -6.74275623e-01 ...  4.18928953e-01
     4.03442363e-01  3.85721871e-01]
   [-6.78388808e-01 -6.83925229e-01 -6.82991124e-01 ...  3.61854515e-01
     3.46860456e-01  3.26510104e-01]]

  [[-3.27670000e+04 -3.27670000e+04 -3.27670000e+04 ... -3.27670000e+04
    -3.27670000e+04 -3.27670000e+04]
   [-3.27670000e+04 -3.27670000e+04 -3.27670000e+04 ... -3.27670000e+04
    -3.27670000e+04 -3.27670000e+04]
   [-3.27670000e+04 -3.27670000e+04 -3.27670000e+04 ... -3.27670000e+04
    -3.27670000e+04 -3.27670000e+04]
   ...
   

In [9]:
with open("/Users/rowenxu/LSTM_EA_Monsoon/Data/output/Nino34_data.txt",'w') as s:
    s.write("T\n")
    for i in range(len(time)):
        for j in range(len(latitude)):
            for k in range(len(longitude)):
                s.write(str(T_norm_nino34[i,0,j,k])+" ")
            s.write("\n")
        s.write("\n")
    